In [12]:
import numpy as np
import pandas as pd
import pickle
import string
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD



import json

In [13]:
intents = json.loads(open('intents.json').read())

In [14]:
# Download NLTK resources
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:

words = []
classes = []
documents = []
context = []
ignore = list(map(lambda x: x, string.punctuation))

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = word_tokenize(pattern.lower())
        words.extend(word_list)
        documents.append((word_list, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

        if intent['context'] not in context:
            context.append(intent['context'])


In [16]:
# Lemmatizer
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore and word not in stop_words]

words = sorted(set(words))


In [17]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
pickle.dump(context,open('context.pkl','wb'))



In [18]:
import pandas as pd

train_data = []
output_empty = [0] * len(classes)

for document in documents:
    bag_of_words = []
    pattern_words = document[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for word in words:
        bag_of_words.append(1) if word in pattern_words else bag_of_words.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    train_data.append({'bow': bag_of_words, 'output_row': output_row})

train_df = pd.DataFrame(train_data)
train_df
train_data = np.array(train_df)
train_pattern = list(train_data[:,0])
train_intents = list(train_data[:,1])
print("Training data created")
print(train_pattern[15])
print(len(train_pattern[0]), "train_pattern",train_pattern[0])

Training data created
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
65 train_pattern [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
# Model
model = Sequential()
model.add(Dense(128,input_shape=(len(train_pattern[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_intents[0]), activation = 'softmax'))

sgd = SGD(learning_rate=0.01,momentum= 0.9,nesterov=True)

model.compile(loss= 'categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


c:\Users\Dell2\Desktop\Project_Acm\chatbot_env\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
history = model.fit(np.array(train_pattern),np.array(train_intents),epochs=200,batch_size=5,verbose= 1)


Epoch 1/200


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0962 - loss: 2.2024
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2956 - loss: 2.1261 
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2570 - loss: 2.1120 
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3487 - loss: 1.9880 
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4730 - loss: 1.9478 
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2989 - loss: 1.9471 
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3981 - loss: 1.8524 
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5079 - loss: 1.7228 
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5966 - loss: 1.5367 
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6486 - loss: 1.5161 
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5264 - loss: 1.3351 
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0

In [21]:
model.save("chat_bot.keras",history)
print("Model Created and Saved Successfully")

Model Created and Saved Successfully
